# Dependences

In [15]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")
include("../../src/extract_info.jl")

using ProgressMeter
# include("../../src/run_event.jl")

# Model

In [16]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = "",
        d_saved = 0.5
    ) 
)

  0.000001 seconds (4 allocations: 144 bytes)


ModelSet(TimeModel(150000.0, 0.5, 100, 50), InputModel(0.8, "../../data/init/Sphere"), OutputModel("Test_1", "", 0.5))

In [17]:
# Run Modelusing Cthulhu
Par1 ,Par2 = Cubic(0.05,2.0,3.0), ContractilePar(0.267,pi/4,0.08,1.0)
# Par1 ,Par2 = Cubic(4.2,2.0,3.0), ContractilePar(0.267,pi/4,0.08,1.0)
size_agg = 15

15

In [18]:
# Run only one aggregate
agg = nothing
agg = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar(Par1, Par2),
        Float32.(readdlm("../../data/init/Sphere/$(size_agg).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)

Aggregate(AggType[AggType("HEK_1", InteractionPar(Cubic{Float64}(0.05, 2.0, 3.0), ContractilePar{Float64}(0.267, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})], AggIndex([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], ["HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"  …  "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1", "HEK_1"]), Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], AggGeometry(Float32[15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27  …  15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27, 15.27], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], (-14.5f0, 14.5f0)), AggSimulation(AggLimit(29.0f0, Bool[0]), AggParameter(Cubic{C

In [19]:
function run_test(agg::Aggregate, model::ModelSet, title::String,save_xyz, save_dat)

    open(model.Output.name_output*".xyz", "w") do f end

    open(model.Output.name_output*".cart.dat", "w") do f
        write(f, "# This file was created $(Dates.format(Dates.now(), "e, dd u yyyy HH:MM:SS")) \n")
        write(f, "# Created by CellAggregate.jl \n")
        write(f, "# Based on Cartesian Coordinates \n")
        write(f, "@    xaxis label \"x position of each cell\"  \n")
        write(f, "@    yaxis label \"y position of each cell\"  \n")
    end
    open(model.Output.name_output*".sph.dat", "w") do f
        write(f, "# This file was created $(Dates.format(Dates.now(), "e, dd u yyyy HH:MM:SS")) \n")
        write(f, "# Created by CellAggregate.jl \n")
        write(f, "# Based on Spherical Coordinates \n")
        write(f, "@    xaxis label \"x position of each cell\"  \n")
        write(f, "@    yaxis label \"y position of each cell\"  \n")
    end
    open(model.Output.name_output*".nw.dat", "w") do f
        write(f, "# This file was created $(Dates.format(Dates.now(), "e, dd u yyyy HH:MM:SS")) \n")
        write(f, "# Created by CellAggregate.jl \n")
        write(f, "# Based on Neck Width Dim of the Aggregate \n")
        write(f, "! | t | Neck | Width   \n")
    end

    @showprogress "$(title)..." for t=0:Int(model.Time.tₛᵢₘ/model.Time.dt)

        if agg.Simulation.Limit.break_sim == CuArray([false])
            if t%(model.Time.nₖₙₙ) == 0
                # println("▲ Neighbor")
                threads=(100)
                @cuda(
                    threads=threads,
                    blocks=cld.(size(agg.Position,1),threads),
                    dist_kernel!(
                        agg.Simulation.Neighbor.idx_red,
                        agg.Simulation.Neighbor.idx_cont,
                        agg.Simulation.Neighbor.idx_sum,
                        agg.Simulation.Neighbor.dist,
                        agg.Position,
                        agg.Simulation.Parameter.Force.rₘₐₓ
                    )
                ) 
            end

            t_max_min=1
            # println(t)
            # CUDA.@time 
            threads=(64,3)
            @cuda(
                threads = threads,
                blocks = cld.(size(agg.Position,),threads),
                sum_force!(
                    agg.Position,
                    agg.Simulation.Force.F,
                    agg.Simulation.Force.Pol,
                    agg.Simulation.Force.dPol,
                    agg.Simulation.Force.N_i,
                    agg.Simulation.Neighbor.idx_sum,
                    agg.Simulation.Neighbor.idx_red,
                    agg.Simulation.Parameter.Force,
                    agg.Simulation.Parameter.Contractile.fₚ,
                    agg.Simulation.Parameter.Contractile.ψₜ,
                    agg.Simulation.Parameter.Contractile.ψₘ,
                    agg.Simulation.Parameter.Contractile.ω,
                    model.Time.dt,
                    agg.Simulation.Limit.max_grid,
                    agg.Simulation.Limit.break_sim
                )
            )
    
            if t%Int(model.Time.tₛᵢₘ/model.Time.nₛₐᵥₑ/model.Time.dt) == 0
                # println("▲ Save")
                push!(agg.Simulation.Output.time,t*model.Time.dt)

                max_min = max_min_agg(agg.Position,40,10,2)
                push!(agg.Simulation.Output.xy_data,max_min)

                sph_data = cart_to_sph(max_min)
                push!(agg.Simulation.Output.θr_data,sph_data)

                neck_width = neck_width_agg(sph_data)
                push!(agg.Simulation.Output.neck_data,[neck_width[2]])
                push!(agg.Simulation.Output.width_data,[neck_width[1]])

                if save_xyz
                    open(model.Output.name_output*".xyz", "a") do f
                        write(f, "$(size(agg.Position, 1))\n")
                        write(f, "t=$(t*model.Time.dt)\n")
                        writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Position)), ' ')
                    end
                end
                if save_dat
                    open(model.Output.name_output*".cart.dat", "a") do f
                        write(f, "!    t = $(t*model.Time.dt)  \n")
                        writedlm(f,max_min')
                    end

                    open(model.Output.name_output*".sph.dat", "a") do f
                        write(f, "!    t = $(t*model.Time.dt)  \n")
                        writedlm(f,sph_data')
                    end

                end
            end
        end

    end

    if agg.Simulation.Limit.break_sim == CuArray([false])
        agg.Simulation.Output.neck_data  = hcat(agg.Simulation.Output.neck_data...) ./agg.Simulation.Output.width_data[1] .* 2
        agg.Simulation.Output.width_data = hcat(agg.Simulation.Output.width_data...) ./agg.Simulation.Output.width_data[1] .* 2
        open(model.Output.name_output*".nw.dat", "a") do f
            writedlm(f,vcat(
                    hcat(agg.Simulation.Output.time...),
                    hcat(agg.Simulation.Output.neck_data...),
                    hcat(agg.Simulation.Output.width_data...)
                )' ./agg.Simulation.Output.width_data[1]
            )
        end
    elseif agg.Simulation.Limit.break_sim == CuArray([true])
        println("Breaking the Simulation (NaN Value or Bigger Values)")
    end

end

run_test (generic function with 1 method)

In [20]:
run_test(agg, model,"Run One Aggregate", false, false)

Run One Aggregate...   0%|                               |  ETA: 2 days, 10:49:02

Run One Aggregate...   0%|▏                              |  ETA: 0:08:19

Run One Aggregate...   1%|▏                              |  ETA: 0:04:26

Run One Aggregate...   1%|▎                              |  ETA: 0:03:10

Run One Aggregate...   1%|▍                              |  ETA: 0:02:29

...

Run One Aggregate...  99%|██████████████████████████████▋|  ETA: 0:00:00

Run One Aggregate...  99%|██████████████████████████████▊|  ETA: 0:00:00

Run One Aggregate... 100%|██████████████████████████████▉|  ETA: 0:00:00

Run One Aggregate... 100%|███████████████████████████████|  ETA: 0:00:00

Run One Aggregate... 100%|███████████████████████████████| Time: 0:00:34


In [29]:
agg.Position

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -5.46062     0.132846   -7.15011
 -4.47899    -3.34361    -7.9291
 -3.14022   -15.1522      4.99523
 10.0624     -5.75074    -3.43293
 -9.10029    -1.13558    -7.26083
 -1.38625    -8.86768   -11.979
  0.200453   -9.13916    -4.78233
  7.35476     0.452525  -11.0049
 -1.56913    -7.99983    -8.92491
  1.59148    -8.75768    -9.61499
  ⋮                     
  8.12374    -8.80674     7.21409
  1.80557     5.02722     8.39349
 -6.30332    -9.77037     5.56432
  1.12644     2.12805    10.4064
  4.71075    -4.70089    14.2278
 -4.33737   -10.3938      9.63828
 -8.7573     -1.06177     9.38257
 -1.6004     -3.39002    12.5481
  8.23814     2.54549    11.3305

# review

In [21]:
# if sum(isnan.(agg.Position)) + sum(abs.(agg.Position) .> agg.Simulation.Limit.max_value) > 0
#     agg.Simulation.Limit.break_sim = true
#     println("Breaking Simulation")
# end

# if agg.Simulation.Limit.break_sim ==  CuArray([false])
#     threads=(100)
#     @cuda(
#         threads=threads,
#         blocks=cld.(size(agg.Position,1),threads),
#         dist_kernel!(
#             agg.Simulation.Neighbor.idx_red,
#             agg.Simulation.Neighbor.idx_cont,
#             agg.Simulation.Neighbor.idx_sum,
#             agg.Simulation.Neighbor.dist,
#             agg.Position,
#             agg.Simulation.Parameter.Force.rₘₐₓ
#         )
#     ) 

#     for i =1:50
#         threads=(64,3)
#         @cuda(
#             threads = threads,
#             blocks = cld.(size(agg.Position,),threads),
#             sum_force!(
#                 agg.Position,
#                 agg.Simulation.Force.F,
#                 agg.Simulation.Force.Pol,
#                 agg.Simulation.Force.dPol,
#                 agg.Simulation.Force.N_i,
#                 agg.Simulation.Neighbor.idx_sum,
#                 agg.Simulation.Neighbor.idx_red,
#                 agg.Simulation.Parameter.Force,
#                 agg.Simulation.Parameter.Contractile.fₚ,
#                 agg.Simulation.Parameter.Contractile.ψₜ,
#                 agg.Simulation.Parameter.Contractile.ψₘ,
#                 agg.Simulation.Parameter.Contractile.ω,
#                 model.Time.dt,
#                 agg.Simulation.Limit.max_grid,
#                 agg.Simulation.Limit.break_sim
#             )
#         )
#     end
# elseif agg.Simulation.Limit.break_sim ==  CuArray([true])
#     println("Breaking the simulation")
# end


In [22]:
# A = zeros(100,3) 
# A[50,2] = NaN
# A = A |> cu

# B = [false] |> cu

In [23]:
# A = agg.Position
# B = agg.Simulation.Limit.break_sim

In [24]:
# function kernel!(pos,max_grid,break_sim)

#     i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
#     k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

#     if i <= size(pos, 1) && k <= size(pos, 2)
#         if pos[i,k] > max_grid || isnan(pos[i,k])
#             break_sim[1,1] = true
#         end
#     end
#     return nothing
# end

In [25]:
# threads=(64,3)
# @cuda(                
#     threads = threads,
#     blocks = cld.(size(agg.Position),threads),
#     kernel!(
#         agg.Position,
#         agg.Simulation.Limit.max_grid,
#         agg.Simulation.Limit.break_sim)
# )
# B

In [26]:
# agg.Simulation.Limit.break_sim

In [27]:
# agg.Simulation.Limit.break_sim == CuArray([false])

In [28]:
# if agg.Simulation.Limit.break_sim == CuArray([false])
#     println("its false")
# else
#     println("its true")
# end